# CSCI2000U - Final Project


## Introduction

We chose this dataset pulled off of Kaggle because it pertains to movies and tv series, which everyone in the group is familiar with, and also has an interest in. Going through other datasets, we were often disappointed either by the lack of data, or the range of categories that are covered within the dataset. This dataset was able to do what we needed: have lots of data we can parse, and have lots of columns we can work with.

We initially looked at using sets such a Utility Cut permits or chemical tracking data for the city of Toronto, but they either didn't have enough data to reasonably work with or weren't interesting to everyone in the group.

Due to this, we decided to go with the IMDB's most popular films and series set.



### These are the list of questions we wish to answer, and expand upon in our analysis:
   - How does the amount of episodes in a tv show effect the ratings? For shows with a greater amount of episodes,
        are the average rating worse, or better than shows with fewer?
   - What are considered popular genres over the years? Do the ratings correlate? 
   - What impact does mature content make on the ratings of movies/shows the most?
   - Do TV series or movies perform better? Based on the year of publish, does this make an impact or change the ratings?
   - ADDITIONAL QUESTION GOES HERE

### Members and contributions:

Ashar Izhar ->

Bridget Green ->

Cameron Millar ->

Julian Finley -> 


## Description of Data

The IMDB's most popular film and series dataset is a list described by its name. In this dataset we can see a range of movies and films from the early 20th century to the current year, with information about the movie such as duration, genre, number of episodes, as well as user engagement information such as ratings and votes. All these datapoints can prove to be useful when diving into the analysis of said data.

The data was collected by the dataset creator using web scraping directly from the IMDB website, and then compiled within a csv file for ease of use.

Full credits go towards the compiler of the dataset; Mazen Ramadan, with a link to the set itself: https://www.kaggle.com/mazenramadan/imdb-most-popular-films-and-series

Information from this dataset was retrieved from the original source: https://www.imdb.com/


## Analysis of the data

The data is organized in such a manner thatis easy to identify their use case and how they can be manipulated. Below is the list of columns within the dataset, and their description:

    Name: The title of the film or tv series
    Date: The date by year
    Rate: The IMDB rating given for the title
    Votes: The number of user votes which have contributed the the votes column
    Genre: The genre(s) of the film or tv series. Multiple can be listed per title
    Duration: The running length in minutes for the film or tv series per episode.
    Type: The indicator on whether this is a film or tv series
    Certificate: The audiance and/or age rating for the title(TV-Y, PG, PG-13, etc)
    Episodes: If this is a tv series, the number of episodes present is recorded
    Adult oriented content present(from mild, moderate, severe):
        -Nudity
        -Violence
        -Profanity
        -Alcohol
        -Frightening
      
The data quality holds up, and any missing data present is due to it either being non-existant(i.e., no adult content being in an all ages movie), or no votes/ratings(no user engagement with the film or tv series).

Data that would be prudent to double check for integrity and cleaning would be:
    
    -Completely empty columns
    -Missing or non-numerical dates
    -Removal of 'No Rates' when doing analysis on the ratings column 
    -Removal of 'No Votes' when doing analysis on the votes column
    -If the genre column has more than one genre, make sure all are taken into account for their appropriate use cases.
    -Removal of 'None' when doing analysis on the duration column
    -When doing analysis on the type column, specify directly which will be worked with
    -Remove all non-numerical from episode column if working with tv series
    -When working will adult content, remove 'No Rate' entries

## Exploratory Data Analysis

## Potential Data Science

## Conclusion